In [1]:
!pip install scikit-learn pandas numpy


In [2]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
FEATURE_COLS = [
    "PTS_8_game_avg", "PTS_8_game_slope",
    "REB_8_game_avg", "REB_8_game_slope",
    "AST_8_game_avg", "AST_8_game_slope",
    "opp_E_OFF_RATING", "opp_E_DEF_RATING",
    "opp_E_NET_RATING", "opp_REB_PCT"
]
TARGET_COLS = ["PTS", "REB", "AST"]
N_JOBS = 4

# ─── Hyperparameter grid ──────────────────────────────────────────────────
param_grid = {
    "alpha": [0.1, 1.0, 10.0],
    "l1_ratio": [0.1, 0.5, 0.9]
}

# ─── Single-fold runner (returns per-target metrics) ───────────────────────
def run_elasticnet_fold(fold_idx, alpha, l1_ratio):
    train_df = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    test_df  = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv", parse_dates=["GAME_DATE"])

    X_train = train_df[FEATURE_COLS].values
    y_train = train_df[TARGET_COLS].values
    X_test  = test_df[FEATURE_COLS].values
    y_test  = test_df[TARGET_COLS].values

    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit separate ElasticNet model for each target
    metrics = {"filters": None, "kernel_size": None, "epochs": None, "fold": fold_idx}
    for i, target in enumerate(TARGET_COLS):
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000)
        model.fit(X_train, y_train[:, i])
        preds = model.predict(X_test)

        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_test[:, i], preds))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_test[:, i], preds)
        metrics[f"{target}_r2"]   = r2_score(y_test[:, i], preds)
    return metrics

# ─── Run across all folds and hyperparameter combos ────────────────────────
all_results = []
for alpha in param_grid["alpha"]:
    for l1_ratio in param_grid["l1_ratio"]:
        fold_metrics = Parallel(n_jobs=N_JOBS)(
            delayed(run_elasticnet_fold)(fold, alpha, l1_ratio) for fold in range(1, 11)
        )
        all_results.extend(fold_metrics)

# ─── Compile and display results ────────────────────────────────────────────
final_df = pd.DataFrame(all_results)
ordered_cols = [
    "filters", "kernel_size", "epochs", "fold",
    "PTS_rmse", "PTS_mae", "PTS_r2",
    "REB_rmse", "REB_mae", "REB_r2",
    "AST_rmse", "AST_mae", "AST_r2"
]
final_df = final_df[ordered_cols]
print(final_df.to_string(index=False))


filters kernel_size epochs  fold  PTS_rmse  PTS_mae    PTS_r2  REB_rmse  REB_mae        REB_r2  AST_rmse  AST_mae    AST_r2
   None        None   None     1  7.891726 6.167245  0.246299  2.938989 2.265935  2.524689e-01  2.282830 1.706242  0.408456
   None        None   None     2  6.797440 5.374315  0.390567  2.710957 2.073385  4.143342e-01  2.187118 1.617015  0.437521
   None        None   None     3  6.674905 5.319503  0.381639  2.710064 2.063355  4.199387e-01  1.955105 1.493529  0.482371
   None        None   None     4  6.948832 5.479094  0.393522  2.734637 2.114550  4.383397e-01  2.121987 1.589732  0.428049
   None        None   None     5  6.665395 5.240014  0.424950  2.684797 2.036905  4.692965e-01  2.082037 1.578842  0.466220
   None        None   None     6  6.902367 5.388315  0.421292  2.728937 2.093070  4.331840e-01  2.088139 1.594214  0.455386
   None        None   None     7  6.904599 5.472489  0.375324  2.833329 2.178001  3.854704e-01  2.001363 1.539984  0.474832
   None 